In [33]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression

from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [137]:
# load players.csv to identify players by their name
players = pd.read_csv("C:/Users/19012/Documents/BDB 2023/players.csv")

# load data to calculate distances from players on the field (and also the distance from the football)
scout = pd.read_csv("C:/Users/19012/Documents/BDB 2023/pffScoutingData.csv")
data = pd.read_csv("C:/Users/19012/Documents/BDB 2023/week1.csv")
plays = pd.read_csv("C:/Users/19012/Documents/BDB 2023/plays.csv")

# merge scout data onto the tracking data 
data = data.merge(scout, how='left')

In [3]:
scout.pff_role.unique()

array(['Pass', 'Pass Route', 'Pass Block', 'Pass Rush', 'Coverage'],
      dtype=object)

In [4]:
scout.head()

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0


In [136]:
scout[(scout.gameId==2021090900)&(scout.playId==137)]#&(scout.nflId==46081.0)]

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
22,2021090900,137,35441,Pass Rush,DRT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2021090900,137,35454,Pass Rush,ROLB,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2021090900,137,37082,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,35454.0,PT,0.0
25,2021090900,137,38588,Coverage,RLB,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2021090900,137,40074,Pass Rush,LEO,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,2021090900,137,41915,Coverage,SCBoL,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,2021090900,137,42347,Pass Route,SLWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,2021090900,137,42654,Pass Block,RT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,40074.0,PP,0.0
30,2021090900,137,43293,Pass Route,RWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2021090900,137,43424,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
plays[(plays.gameId==2021090900)&(plays.playId==137)]#&(scout.nflId==46081.0)]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,13:18,0,0,C,NaN,28,28,NaN,NaN,NaN,NaN,NaN,NaN,108.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 4 LB, 3 DB",TRADITIONAL,0,Cover-3,Zone


In [5]:
data.head()

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data.event.unique()

array(['None', 'ball_snap', 'autoevent_passforward', 'pass_forward',
       'autoevent_ballsnap', 'line_set', 'play_action', 'pass_arrived',
       'autoevent_passinterrupted', 'fumble', 'fumble_offense_recovered',
       'qb_sack', 'run', 'man_in_motion', 'pass_outcome_caught',
       'pass_outcome_incomplete', 'pass_tipped', 'qb_strip_sack', 'shift',
       'first_contact', 'huddle_break_offense', 'lateral', 'handoff'],
      dtype=object)

In [17]:
data.pff_positionLinedUp.unique()

array(['QB', 'TE-L', 'LWR', 'HB-R', 'C', 'RWR', 'LEO', 'LT', 'ROLB', 'LG',
       'RCB', 'SLWR', 'SCBR', 'DRT', 'FS', 'RG', 'LILB', 'RT', 'LCB',
       'RE', nan, 'RLB', 'SCBoL', 'SRoWR', 'DLT', 'SRiWR', 'SCBiL', 'REO',
       'RILB', 'SSR', 'LLB', 'LE', 'LOLB', 'NLT', 'TE-oR', 'TE-iR', 'HB',
       'FSR', 'MLB', 'HB-L', 'TE-R', 'SRWR', 'SCBL', 'NT', 'FSL', 'SSL',
       'NRT', 'SS', 'SLoWR', 'SLiWR', 'SCBiR', 'SCBoR', 'TE-oL', 'TE-iL',
       'FB-L', 'FB', 'FB-R'], dtype=object)

In [20]:
data[(data['pff_positionLinedUp']=='DLT')&(data['gameId']==2021090900)&(data['playId']==137)]

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
1396,2021090900,137,46081.0,1,2021-09-10T00:28:09.900,50.0,TB,left,108.62,24.99,0.00,0.01,0.02,168.25,356.29,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1397,2021090900,137,46081.0,2,2021-09-10T00:28:10.000,50.0,TB,left,108.62,24.99,0.00,0.01,0.00,168.25,355.98,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1398,2021090900,137,46081.0,3,2021-09-10T00:28:10.100,50.0,TB,left,108.61,24.98,0.00,0.00,0.02,168.25,350.87,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1399,2021090900,137,46081.0,4,2021-09-10T00:28:10.200,50.0,TB,left,108.61,24.98,0.00,0.00,0.00,168.25,348.56,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1400,2021090900,137,46081.0,5,2021-09-10T00:28:10.300,50.0,TB,left,108.61,24.98,0.00,0.00,0.00,168.25,337.40,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1401,2021090900,137,46081.0,6,2021-09-10T00:28:10.400,50.0,TB,left,108.55,24.96,0.00,0.02,0.06,232.84,266.47,autoevent_ballsnap,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1402,2021090900,137,46081.0,7,2021-09-10T00:28:10.500,50.0,TB,left,108.48,24.99,0.02,0.11,0.07,195.54,277.96,ball_snap,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1403,2021090900,137,46081.0,8,2021-09-10T00:28:10.600,50.0,TB,left,108.48,24.99,0.03,0.09,0.00,195.54,278.88,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1404,2021090900,137,46081.0,9,2021-09-10T00:28:10.700,50.0,TB,left,108.46,25.08,0.07,0.18,0.09,121.37,317.69,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1405,2021090900,137,46081.0,10,2021-09-10T00:28:10.800,50.0,TB,left,108.55,25.23,0.23,0.84,0.18,123.34,11.55,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Closest Player

Pick an arbitrary play and find the closest player to each observation

In [23]:
# create an empty output dataframe to append data into 
df_presnap = pd.DataFrame()

# loop through each game 
for gid in data['gameId'].unique():
  # subset the data down to one game
  one_game = data.loc[data['gameId']==gid]

  # loop through each play
  for pid in one_game['playId'].unique():

    # subset game data down to one play 
    one_play = one_game.loc[one_game['playId']==pid]
    
    # loop through each frame
    # isolate the first frame of data only (you could also loop through all frames if you so choose. this example is supposed to be a short demo)
    one_frame = one_play.loc[one_play['frameId']==1]

    # reset index (saving the original index as a column) and set new index as player ID 
    one_frame = one_frame.reset_index().set_index('nflId')

    # turn PFF player positioning info into unique positions given the play (for modeling purposes)
    one_frame['pff_positionLinedUp'] = one_frame['pff_positionLinedUp'] + one_frame.groupby('pff_positionLinedUp', as_index=False).cumcount().astype(str).str.replace('0', '')
    
    # get pairwise distance, turn it into a pairwise matrix, set it in a pandas dataframe with index as nflId and columns as positions 
    _df = pd.DataFrame(squareform(pdist(one_frame.loc[:, ['x','y']])), index=one_frame.index, columns=one_frame['pff_positionLinedUp'].unique())

    # concat pairwise matrix column-wise onto original one_frame data
    one_frame = pd.concat([one_frame,_df], axis=1).rename(columns={np.nan:'dist_from_ball'})

    # change index back to nflId column, set the column "index" to the true index 
    one_frame = one_frame.reset_index().set_index('index')

    # clear the index name for prettyfication 
    one_frame.index.name = None

    # append to output dataframe
    df_presnap = df_presnap.append(one_frame)

# fill the nulled sparse positions with -1, indicating that position was not apparent on a given play and/or frame
positions = [x for x in df_presnap.columns.values if x not in one_game.columns.values]
df_presnap.loc[:, positions] = df_presnap.loc[:, positions].fillna(-1) 

In [22]:
_df

,DRT,ROLB,LT,RLB,LEO,SCBoL,SLWR,RT,RWR,QB,SRoWR,DLT,LG,LCB,SRiWR,SCBiL,RCB,RG,LWR,FS,C,SCBR,NaN
nflId,,,,,,,,,,,,,,,,,,,,,,,
35441.0,0.000000,1.654841,2.721066,3.276110,7.090007,13.060065,7.649660,6.378409,23.474669,6.839415,15.173925,4.493951,2.742353,24.455707,10.768570,8.658891,11.989600,4.304428,13.452498,15.389256,4.379612,5.624278,2.903188
35454.0,1.654841,0.000000,2.201931,4.225376,8.594562,14.646365,6.021993,7.452141,24.844090,6.947179,16.463004,5.707933,3.220745,26.096392,12.080472,10.311144,10.649178,5.479599,11.864948,16.492814,5.392263,4.422420,4.157403
37082.0,2.721066,2.201931,0.000000,5.963699,7.971198,14.158619,6.867001,6.066012,23.666392,4.786084,15.192689,4.676003,1.721308,25.855263,10.914124,10.378102,12.178690,4.410726,12.750392,18.105850,4.091015,6.301785,3.448028
38588.0,3.276110,4.225376,5.963699,0.000000,7.985719,13.112109,9.050271,8.546075,24.336857,9.971966,16.428807,6.611392,5.881853,23.757157,12.104776,8.123257,11.886076,6.526638,14.306824,12.268028,6.856566,5.814258,5.261654
40074.0,7.090007,8.594562,7.971198,7.985719,0.000000,6.188093,14.557531,3.300682,16.483571,7.580673,8.443329,3.392492,6.288474,17.929824,4.169904,3.587339,19.053044,3.659030,20.442664,15.510313,4.178086,12.666432,4.567154
41915.0,13.060065,14.646365,14.158619,13.112109,6.188093,0.000000,20.658645,8.885989,11.728427,13.152053,6.043153,9.551466,12.475015,11.797851,5.183532,5.112817,24.807438,9.820489,26.507754,16.022000,10.290389,18.452387,10.739162
42347.0,7.649660,6.021993,6.867001,9.050271,14.557531,20.658645,0.000000,12.933012,30.532114,10.784456,22.059476,11.450210,8.560093,32.103466,17.770003,16.288355,5.923799,11.195035,5.904253,20.355277,10.940005,3.770332,10.017045
42654.0,6.378409,7.452141,6.066012,8.546075,3.300682,8.885989,12.933012,0.000000,17.602227,4.345216,9.127196,1.951333,4.393313,20.662955,4.883380,6.847226,18.076709,2.091220,18.810768,18.157976,2.060413,11.860059,3.491819
43293.0,23.474669,24.844090,23.666392,24.336857,16.483571,11.728427,30.532114,17.602227,0.000000,20.668878,8.494616,19.152016,21.975304,11.441927,12.778302,16.746238,35.460539,19.393535,36.412781,27.087977,19.598441,29.096189,20.686865


In [8]:
# LE distances from QB and ball
df_presnap[df_presnap['pff_positionLinedUp']=='LEO'][['nflId', 'pff_positionLinedUp', 'gameId', 'playId','QB', 'dist_from_ball']]

,nflId,pff_positionLinedUp,gameId,playId,QB,dist_from_ball
258,41263.0,LEO,2021090900,97,8.072329,5.391030
1137,40074.0,LEO,2021090900,137,7.580673,4.567154
3542,35441.0,LEO,2021090900,349,8.367795,6.651834
4528,41263.0,LEO,2021090900,410,8.638154,6.581284
5838,41263.0,LEO,2021090900,456,8.346406,6.085820
...,...,...,...,...,...,...
1111647,37240.0,LEO,2021091300,4586,6.922254,4.692206
1114218,47889.0,LEO,2021091300,4641,7.590026,4.730592
1115160,47889.0,LEO,2021091300,4691,7.962073,5.701965
1116049,47889.0,LEO,2021091300,4736,7.769492,5.154309


In [9]:
# let's define "pressure allowed" as any time a player allowed a hit, sack, or hurry as defined by PFF's scouting data
df_presnap['pressure_allowed'] = df_presnap.loc[:, ['pff_hitAllowed', 'pff_hurryAllowed', 'pff_sackAllowed']].sum(axis=1)
df_presnap.head()

,nflId,gameId,playId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,QB,TE-L,LWR,HB-R,C,RWR,LEO,LT,ROLB,LG,ROLB1,RCB,SLWR,SCBR,DRT,FS,RG,LILB,RT,LCB,LILB1,RE,dist_from_ball,RLB,SCBoL,SRoWR,DLT,SRiWR,SCBiL,REO,RILB,SCBiL1,SSR,LLB,LE,LOLB,NLT,TE-oR,LLB1,TE-iR,HB,FSR,MLB,HB-L,TE-R,SRWR,SCBL,NT,FSL,SSL,NRT,SS,SLoWR,SLiWR,SCBiR,SCBoR,LEO1,TE-oL,TE-iL,LOLB1,DRT1,SCBL1,LCB1,SCBR1,SCBoL1,SCBiR1,FS1,FB-L,FB,LLB2,RLB1,RILB1,MLB1,ROLB2,SCBoR1,RCB1,FSL1,FB-R,FSR1,LOLB2,REO1,DLT1,FSR2,SS1,pressure_allowed
0,25511.0,2021090900,97,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,5.938097,12.927444,1.973449,4.417080,9.080666,8.072329,4.627883,10.532260,4.421086,8.698988,14.505423,12.713473,15.138246,5.773768,21.060888,3.981281,10.684779,4.427640,9.799765,6.384293,6.406942,5.153145,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
43,35481.0,2021090900,97,1,2021-09-10T00:26:31.100,87.0,TB,right,41.08,29.15,0.31,0.43,0.03,35.96,238.77,None,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.938097,0.000000,7.590534,7.746206,5.295508,12.993325,10.763777,2.308354,4.608047,3.596804,3.541327,8.738884,7.634461,9.282958,4.668212,17.614278,6.528859,9.660274,7.790314,13.105373,7.731093,3.600347,5.431179,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
86,35634.0,2021090900,97,1,2021-09-10T00:26:31.100,81.0,TB,right,40.99,36.74,0.00,0.00,0.00,113.75,325.69,None,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.927444,7.590534,0.000000,14.885325,12.825327,20.575298,18.236516,9.865906,5.064000,11.121511,7.978371,2.627946,0.900222,3.660833,11.862141,19.966477,14.116551,15.768339,15.380832,20.623312,15.106886,10.425685,12.845813,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
129,39985.0,2021090900,97,1,2021-09-10T00:26:31.100,25.0,TB,right,37.54,22.26,0.11,0.07,0.02,91.78,132.28,None,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.973449,7.746206,14.885325,0.000000,4.945068,7.538289,7.080113,6.125112,12.284340,5.568492,10.191766,16.406907,14.683123,17.009174,6.592367,21.661683,3.897602,10.833467,3.720497,8.449953,6.181691,7.567734,5.666816,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
172,40151.0,2021090900,97,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,0.00,0.00,0.01,45.36,206.92,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,4.417080,5.295508,12.825327,4.945068,0.000000,7.781857,5.473874,2.987189,8.914084,1.705579,6.142524,13.643039,12.920163,14.003517,1.741551,16.760251,1.540519,6.289777,2.805887,7.812324,2.590174,3.093558,0.743303,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1

In [15]:
# subset the data to only players on the "pass rush" to see distances pre-snap
# this will help determine when a player is being blocked
# original idea was to use the average length of player's arm x2 (1.88 yards); however, in this observation, the
# player is already within that threshold on 2 players pre-snap
# Use 0.94 yards to account for the fact that players engage in blocks with arms bent
model_data = df_presnap.loc[df_presnap['pff_role']=='Pass Rush']
model_data.head()

,nflId,gameId,playId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,QB,TE-L,LWR,HB-R,C,RWR,LEO,LT,ROLB,LG,ROLB1,RCB,SLWR,SCBR,DRT,FS,RG,LILB,RT,LCB,LILB1,RE,dist_from_ball,RLB,SCBoL,SRoWR,DLT,SRiWR,SCBiL,REO,RILB,SCBiL1,SSR,LLB,LE,LOLB,NLT,TE-oR,LLB1,TE-iR,HB,FSR,MLB,HB-L,TE-R,SRWR,SCBL,NT,FSL,SSL,NRT,SS,SLoWR,SLiWR,SCBiR,SCBoR,LEO1,TE-oL,TE-iL,LOLB1,DRT1,SCBL1,LCB1,SCBR1,SCBoL1,SCBiR1,FS1,FB-L,FB,LLB2,RLB1,RILB1,MLB1,ROLB2,SCBoR1,RCB1,FSL1,FB-R,FSR1,LOLB2,REO1,DLT1,FSR2,SS1,pressure_allowed
258,41263.0,2021090900,97,1,2021-09-10T00:26:31.100,90.0,DAL,right,43.66,18.70,0.45,0.92,0.05,308.77,337.59,None,Pass Rush,LEO,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.072329,10.763777,18.236516,7.080113,5.473874,3.003764,0.000000,8.457435,13.994892,7.166987,10.959530,18.850767,18.370229,19.098482,6.442802,17.253420,4.523549,6.236986,3.657280,2.435262,3.180393,7.970056,5.391030,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
344,42403.0,2021090900,97,1,2021-09-10T00:26:31.100,94.0,DAL,right,44.03,32.69,0.00,0.00,0.01,265.97,215.94,None,Pass Rush,ROLB,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.532260,4.608047,5.064000,12.284340,8.914084,16.623189,13.994892,6.307963,0.000000,7.331255,3.120657,4.889908,5.636000,5.111653,7.570740,15.524642,10.393060,10.796342,11.690308,16.428551,10.814735,6.029594,8.720929,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
602,44955.0,2021090900,97,1,2021-09-10T00:26:31.100,91.0,DAL,right,43.47,25.14,0.00,0.00,0.01,43.02,104.69,None,Pass Rush,DRT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.773768,4.668212,11.862141,6.592367,1.741551,9.068302,6.442802,2.597865,7.570740,1.606487,4.617878,12.410016,12.063200,12.681897,0.000000,15.287675,3.240772,5.421042,4.435178,8.870051,3.268822,1.545736,1.229837,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
860,53441.0,2021090900,97,1,2021-09-10T00:26:31.100,11.0,DAL,right,43.71,21.88,0.00,0.00,0.01,276.51,290.12,None,Pass Rush,LILB1,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.384293,7.731093,15.106886,6.181691,2.590174,5.938527,3.180393,5.455795,10.814735,4.169652,7.789865,15.671544,15.275143,15.919614,3.268822,15.856059,2.409689,4.740137,2.612279,5.613929,0.000000,4.790042,2.299935,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
903,53504.0,2021090900,97,1,2021-09-10T00:26:31.100,97.0,DAL,right,43.69,26.67,0.01,0.01,0.00,222.44,328.69,None,Pass Rush,RE,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.406942,3.600347,10.425685,7.567734,3.093558,10.613802,7.970056,2.151279,6.029594,2.011890,3.084234,10.881838,10.675322,11.139488,1.545736,14.924600,4.634059,6.065130,5.886230,10.401735,4.790042,0.000000,2.740401,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1

In [10]:
# subset the data to only players on the "pass block" 
model_data = df_presnap.loc[df_presnap['pff_role']=='Pass Block']
model_data.head()

,nflId,gameId,playId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,QB,TE-L,LWR,HB-R,C,RWR,LEO,LT,ROLB,LG,ROLB1,RCB,SLWR,SCBR,DRT,FS,RG,LILB,RT,LCB,LILB1,RE,dist_from_ball,RLB,SCBoL,SRoWR,DLT,SRiWR,SCBiL,REO,RILB,SCBiL1,SSR,LLB,LE,LOLB,NLT,TE-oR,LLB1,TE-iR,HB,FSR,MLB,HB-L,TE-R,SRWR,SCBL,NT,FSL,SSL,NRT,SS,SLoWR,SLiWR,SCBiR,SCBoR,LEO1,TE-oL,TE-iL,LOLB1,DRT1,SCBL1,LCB1,SCBR1,SCBoL1,SCBiR1,FS1,FB-L,FB,LLB2,RLB1,RILB1,MLB1,ROLB2,SCBoR1,RCB1,FSL1,FB-R,FSR1,LOLB2,REO1,DLT1,FSR2,SS1,pressure_allowed
172,40151.0,2021090900,97,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,0.0,0.0,0.01,45.36,206.92,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,4.417080,5.295508,12.825327,4.945068,0.000000,7.781857,5.473874,2.987189,8.914084,1.705579,6.142524,13.643039,12.920163,14.003517,1.741551,16.760251,1.540519,6.289777,2.805887,7.812324,2.590174,3.093558,0.743303,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
301,42377.0,2021090900,97,1,2021-09-10T00:26:31.100,76.0,TB,right,41.55,26.89,0.0,0.0,0.00,148.02,132.28,None,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,42403.0,PP,0.0,4.627883,2.308354,9.865906,6.125112,2.987189,10.709883,8.457435,0.000000,6.307963,1.293406,4.085890,10.835091,9.937832,11.282491,2.597865,17.055676,4.261983,7.943815,5.544448,10.797541,5.455795,2.151279,3.162183,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
387,42404.0,2021090900,97,1,2021-09-10T00:26:31.100,74.0,TB,right,41.95,25.66,0.0,0.0,0.00,108.14,151.12,None,Pass Block,LG,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,PP,0.0,4.421086,3.596804,11.121511,5.568492,1.705579,9.470190,7.166987,1.293406,7.331255,0.000000,4.759328,11.989316,11.215311,12.385350,1.606487,16.733541,3.076004,7.019095,4.373786,9.517069,4.169652,2.011890,1.888200,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
688,46163.0,2021090900,97,1,2021-09-10T00:26:31.100,65.0,TB,right,41.42,22.63,0.0,0.0,0.00,107.71,216.03,None,Pass Block,RG,NaN,NaN,NaN,0.0,0.0,1.0,0.0,53441.0,SW,0.0,3.981281,6.528859,14.116551,3.897602,1.540519,6.466970,4.523549,4.261983,10.393060,3.076004,7.675220,15.062911,14.152632,15.459913,3.240772,17.811300,0.000000,6.942082,1.298384,6.691487,2.409689,4.634059,2.058665,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
774,52421.0,2021090900,97,1,2021-09-10T00:26:31.100,78.0,TB,right,41.15,21.36,0.0,0.0,0.01,49.72,293.16,None,Pass Block,RT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,53441.0,PP,0.0,4.427640,7.790314,15.380832,3.720497,2.805887,5.241040,3.657280,5.544448,11.690308,4.373786,8.947692,16.358230,15.396532,16.758249,4.435178,18.442977,1.298384,7.352394,0.000000,5.606300,2.612279,5.886230,3.211744,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,

In [11]:
# let's use an extremely simple feature set: a player's XY position and all the distances away from each player + the ball
feats = ['x', 'y'] + positions
feats

['x',
 'y',
 'QB',
 'TE-L',
 'LWR',
 'HB-R',
 'C',
 'RWR',
 'LEO',
 'LT',
 'ROLB',
 'LG',
 'ROLB1',
 'RCB',
 'SLWR',
 'SCBR',
 'DRT',
 'FS',
 'RG',
 'LILB',
 'RT',
 'LCB',
 'LILB1',
 'RE',
 'dist_from_ball',
 'RLB',
 'SCBoL',
 'SRoWR',
 'DLT',
 'SRiWR',
 'SCBiL',
 'REO',
 'RILB',
 'SCBiL1',
 'SSR',
 'LLB',
 'LE',
 'LOLB',
 'NLT',
 'TE-oR',
 'LLB1',
 'TE-iR',
 'HB',
 'FSR',
 'MLB',
 'HB-L',
 'TE-R',
 'SRWR',
 'SCBL',
 'NT',
 'FSL',
 'SSL',
 'NRT',
 'SS',
 'SLoWR',
 'SLiWR',
 'SCBiR',
 'SCBoR',
 'LEO1',
 'TE-oL',
 'TE-iL',
 'LOLB1',
 'DRT1',
 'SCBL1',
 'LCB1',
 'SCBR1',
 'SCBoL1',
 'SCBiR1',
 'FS1',
 'FB-L',
 'FB',
 'LLB2',
 'RLB1',
 'RILB1',
 'MLB1',
 'ROLB2',
 'SCBoR1',
 'RCB1',
 'FSL1',
 'FB-R',
 'FSR1',
 'LOLB2',
 'REO1',
 'DLT1',
 'FSR2',
 'SS1']

In [12]:
# try to predict our new feature "pressure_allowed"
target = 'pressure_allowed'

# 5 fold group K-fold (where we group games together so as to not overfit on any potential game result leakage)
kf = GroupKFold(5)

# make an empty series to contain the new preds
preds = pd.Series()

# loop over the dataset, 80% train data, 20% test data
for train_idx, test_idx in kf.split(model_data, groups=model_data['gameId']):

  # split your train and test data by integer-position (aka 0 = row 0, 1 = row 1, etc)
  # if you aren't using sklearn pipeline or other convenience sklearn functions
  # KFold splits are outputted as integer-positions, not label-positions. so you will need to use .iloc (not .loc)
    train_data = model_data.iloc[train_idx]
    test_data = model_data.iloc[test_idx]
  
  # train a simple logistic regression, default parameters blindly accepted
    model = LogisticRegression()
    model.fit(train_data.loc[:, feats], train_data[target])

  # predict on test set and store predictions to a series
  # remember since we're potentially going from numpy outputs to pandas, we can explicitly set our index to the test_data index for assurance in the concatenation
    _s = pd.DataFrame(model.predict_proba(test_data.loc[:, feats]), index=test_data.index)[1]
  
  # append preds to the preds series output
    preds = preds.append(_s)

# concatenate predictions to model_data dataset
model_data[f"{target}_pred"] = preds  

<ipython-input-12-d7a703b09797>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  preds = pd.Series()
C:\Users\19012\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\19012\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
  

In [13]:
model_data

,nflId,gameId,playId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,QB,TE-L,LWR,HB-R,C,RWR,LEO,LT,ROLB,LG,ROLB1,RCB,SLWR,SCBR,DRT,FS,RG,LILB,RT,LCB,LILB1,RE,dist_from_ball,RLB,SCBoL,SRoWR,DLT,SRiWR,SCBiL,REO,RILB,SCBiL1,SSR,LLB,LE,LOLB,NLT,TE-oR,LLB1,TE-iR,HB,FSR,MLB,HB-L,TE-R,SRWR,SCBL,NT,FSL,SSL,NRT,SS,SLoWR,SLiWR,SCBiR,SCBoR,LEO1,TE-oL,TE-iL,LOLB1,DRT1,SCBL1,LCB1,SCBR1,SCBoL1,SCBiR1,FS1,FB-L,FB,LLB2,RLB1,RILB1,MLB1,ROLB2,SCBoR1,RCB1,FSL1,FB-R,FSR1,LOLB2,REO1,DLT1,FSR2,SS1,pressure_allowed,pressure_allowed_pred
172,40151.0,2021090900,97,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,0.0,0.0,0.01,45.36,206.92,None,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0,4.417080,5.295508,12.825327,4.945068,0.000000,7.781857,5.473874,2.987189,8.914084,1.705579,6.142524,13.643039,12.920163,14.003517,1.741551,16.760251,1.540519,6.289777,2.805887,7.812324,2.590174,3.093558,0.743303,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.041025
301,42377.0,2021090900,97,1,2021-09-10T00:26:31.100,76.0,TB,right,41.55,26.89,0.0,0.0,0.00,148.02,132.28,None,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,42403.0,PP,0.0,4.627883,2.308354,9.865906,6.125112,2.987189,10.709883,8.457435,0.000000,6.307963,1.293406,4.085890,10.835091,9.937832,11.282491,2.597865,17.055676,4.261983,7.943815,5.544448,10.797541,5.455795,2.151279,3.162183,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,1.0,0.077512
387,42404.0,2021090900,97,1,2021-09-10T00:26:31.100,74.0,TB,right,41.95,25.66,0.0,0.0,0.00,108.14,151.12,None,Pass Block,LG,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,PP,0.0,4.421086,3.596804,11.121511,5.568492,1.705579,9.470190,7.166987,1.293406,7.331255,0.000000,4.759328,11.989316,11.215311,12.385350,1.606487,16.733541,3.076004,7.019095,4.373786,9.517069,4.169652,2.011890,1.888200,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.064549
688,46163.0,2021090900,97,1,2021-09-10T00:26:31.100,65.0,TB,right,41.42,22.63,0.0,0.0,0.00,107.71,216.03,None,Pass Block,RG,NaN,NaN,NaN,0.0,0.0,1.0,0.0,53441.0,SW,0.0,3.981281,6.528859,14.116551,3.897602,1.540519,6.466970,4.523549,4.261983,10.393060,3.076004,7.675220,15.062911,14.152632,15.459913,3.240772,17.811300,0.000000,6.942082,1.298384,6.691487,2.409689,4.634059,2.058665,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.0,1.0,0.075593
774,52421.0,2021090900,97,1,2021-09-10T00:26:31.100,78.0,TB,right,41.15,21.36,0.0,0.0,0.01,49.72,293.16,None,Pass Block,RT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,53441.0,PP,0.0,4.427640,7.790314,15.380832,3.720497,2.805887,5.241040,3.657280,5.544448,11.690308,4.

# Follow a Player

In [60]:
# create an empty output dataframe to append data into 
df_presnap = pd.DataFrame()

one_play = data[(data['gameId']==2021090900)&(data['playId']==137)]
# loop through each frame
for fid in one_play['frameId'].unique():

    # subset game data down to one frame
    one_frame = one_play.loc[one_play['frameId']==fid]

    # reset index (saving the original index as a column) and set new index as player ID 
    one_frame = one_frame.reset_index().set_index('nflId')

    # turn PFF player positioning info into unique positions given the play (for modeling purposes)
    one_frame['pff_positionLinedUp'] = one_frame['pff_positionLinedUp'] + one_frame.groupby('pff_positionLinedUp', as_index=False).cumcount().astype(str).str.replace('0', '')

    # get pairwise distance, turn it into a pairwise matrix, set it in a pandas dataframe with index as nflId and columns as positions 
    _df = pd.DataFrame(squareform(pdist(one_frame.loc[:, ['x','y']])), index=one_frame.index, columns=one_frame['pff_positionLinedUp'].unique())

    # concat pairwise matrix column-wise onto original one_frame data
    one_frame = pd.concat([one_frame,_df], axis=1).rename(columns={np.nan:'dist_from_ball'})

    # change index back to nflId column, set the column "index" to the true index 
    one_frame = one_frame.reset_index().set_index('index')

    # clear the index name for prettyfication 
    one_frame.index.name = None

    # append to output dataframe
    df_presnap = df_presnap.append(one_frame)

# fill the nulled sparse positions with -1, indicating that position was not apparent on a given play and/or frame
positions = [x for x in df_presnap.columns.values if x not in one_game.columns.values]
df_presnap.loc[:, positions] = df_presnap.loc[:, positions].fillna(-1) 

In [61]:
# 37 frames, 23 players/ball on field = 851
one_play.shape

(851, 28)

In [63]:
print(one_frame.shape)
one_frame#[one_frame['pff_positionLinedUp']=='DLT']

(23, 51)


,nflId,gameId,playId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,DRT,ROLB,LT,RLB,LEO,SCBoL,SLWR,RT,RWR,QB,SRoWR,DLT,LG,LCB,SRiWR,SCBiL,RCB,RG,LWR,FS,C,SCBR,dist_from_ball
1025,35441.0,2021090900,137,37,2021-09-10T00:28:13.500,93.0,TB,left,112.88,23.98,0.96,1.77,0.12,65.24,333.81,None,Pass Rush,DRT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,2.470081,5.048455,18.831221,3.949291,23.349332,24.033928,3.544926,27.435840,3.554490,24.602349,2.696145,1.840679,29.147357,13.333004,14.515406,23.045791,2.052438,20.664104,27.473960,0.572713,5.087092,8.311691
1062,35454.0,2021090900,137,37,2021-09-10T00:28:13.500,90.0,TB,left,111.06,22.31,2.34,1.10,0.24,104.50,198.07,None,Pass Rush,ROLB,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.470081,0.000000,5.826577,16.365146,6.144949,24.224368,21.563882,5.956979,27.657368,5.422629,25.661631,3.491776,1.778145,29.132781,13.604988,13.470698,20.649804,3.419006,18.345618,25.391436,2.840229,6.343311,5.866660
1099,37082.0,2021090900,137,37,2021-09-10T00:28:13.500,77.0,DAL,left,116.63,20.60,2.55,2.03,0.27,122.32,72.63,None,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,35454.0,PT,0.0,5.048455,5.826577,0.000000,19.698472,7.778805,28.055178,25.045958,6.688513,32.442642,2.771299,29.147454,2.447386,4.100195,34.191158,18.357922,19.234763,22.907034,2.998966,20.091802,30.749081,4.561107,1.214002,9.617198
1136,38588.0,2021090900,137,37,2021-09-10T00:28:13.500,54.0,TB,left,99.70,10.53,3.84,3.54,0.40,46.71,231.12,None,Coverage,RLB,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.831221,16.365146,19.698472,0.000000,22.292905,34.852174,5.347541,22.297776,34.764695,21.043557,36.933812,18.642653,17.526808,34.777207,23.840220,17.931071,5.625122,19.111180,5.762508,15.912970,19.143576,20.826838,10.552369
1173,40074.0,2021090900,137,37,2021-09-10T00:28:13.500,92.0,TB,left,113.76,27.83,1.26,1.88,0.13,193.75,236.46,None,Pass Rush,LEO,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.949291,6.144949,7.778805,22.292905,0.000000,20.357416,27.367298,1.262537,25.246830,5.181245,21.381031,6.038021,5.765501,27.272376,11.488416,14.703078,26.764133,5.207264,24.490563,29.320645,3.982876,7.221330,11.949464
1210,41915.0,2021090900,137,37,2021-09-10T00:28:13.500,58.0,TB,left,103.16,45.21,4.69,3.67,0.47,25.16,346.30,None,Coverage,SCBoL,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.349332,24.224368,28.055178,34.852174,20.357416,0.000000,38.250974,21.560928,7.902531,25.538029,2.396101,25.984736,25.014672,11.024196,11.275181,17.251832,40.428436,25.213917,39.390249,31.740715,23.696010,27.576976,27.875217
1247,42347.0,2021090900,137,37,2021-09-10T00:28:13.500,19.0,DAL,left,95.09,7.82,8.03,1.87,0.81,344.02,247.23,None,Pass Route,SLWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.033928,21.563882,25.045958,5.347541,27.367298,38.250974,0.000000,27.451648,37.190389,26.358219,40.449011,23.960077,22.798465,36.776902,27.642433,21.010409,5.656023,24.407364,8.178319,14.087512,24.372199,26.172799,15.820458
1284,42654.0,2021090900,137,37,2021-09-10T00:28:13.500,71.0,DAL,left,114.72,27.01,0.81,2.00,0.08,265.81,195.99,None,Pass Block,RT,NaN,NaN,NaN,0.0,0.0,0.0,0.0,40074.0,PP,0.0,3.544926,5.956979,6.688513,22.297776,1.262537,21.560928,27.451648,0.000000,26.508331,4.009751,22.539665,5.173867,5.223887,28.533393,12.739329,15.716110,26.586809,4.347781,24.205497,29.990895,3.400132,6.046751,11.823625
1321,43293.0,2021090900,137,37,2021-09-10T00:28:13.500,21.0,DAL,left,95.26,45.01,6.76,1.58,0.69,270.57,274.46,None,Pass Route,RWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.435840,27.657368,32.442642,34.764695,25.246830,7.902531,37.190389,26.508331,0.000000,30.245279,9.560298,30.129661,28.839934,3.144980,14.111899,16.992637,40.372174,29.457013,39.950891,28.059526,27.889692,32.215580,29.942607
1358,43424.0,2021090900,137,37,2021-09-10T00:28:13.500,4.0,DAL,left,116.38,23.36,0.38,1.04,0.03,233

maybe join table to itself on nflId	gameId	playId	frameId and keep pff_positionLinedUp and dist to QB of the right table as opponents position.

In [64]:
# should only be 37 for each frame in which this player participated
df_presnap[df_presnap['pff_positionLinedUp']=='DLT'].shape

(37, 51)

In [81]:
dlt_play = df_presnap[df_presnap['pff_positionLinedUp']=='DLT']
dlt_play = dlt_play.reset_index()
dlt_play

,index,nflId,gameId,playId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,DRT,ROLB,LT,RLB,LEO,SCBoL,SLWR,RT,RWR,QB,SRoWR,DLT,LG,LCB,SRiWR,SCBiL,RCB,RG,LWR,FS,C,SCBR,dist_from_ball
0,1396,46081.0,2021090900,137,1,2021-09-10T00:28:09.900,50.0,TB,left,108.62,24.99,0.00,0.01,0.02,168.25,356.29,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.493951,5.707933,4.676003,6.611392,3.392492,9.551466,11.450210,1.951333,19.152016,4.739325,10.755110,0.0,2.958936,21.318171,6.377437,6.416977,16.355626,0.269258,17.354265,16.764176,0.846286,10.060760,1.591760
1,1397,46081.0,2021090900,137,2,2021-09-10T00:28:10.000,50.0,TB,left,108.62,24.99,0.00,0.01,0.00,168.25,355.98,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.501555,5.709588,4.678130,6.562469,3.123284,9.547366,11.421471,1.951333,19.152016,4.795852,10.759986,0.0,2.958936,21.259541,6.377437,6.411123,16.355626,0.288617,17.374855,16.751579,0.820061,10.009486,1.591760
2,1398,46081.0,2021090900,137,3,2021-09-10T00:28:10.100,50.0,TB,left,108.61,24.98,0.00,0.00,0.02,168.25,350.87,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.317708,5.691546,4.670514,6.492888,3.002416,9.544255,11.373452,1.965426,19.161983,4.853308,10.746916,0.0,2.963866,21.263774,6.386306,6.397163,16.404786,0.183848,17.395451,16.727731,0.822192,9.953216,1.578892
3,1399,46081.0,2021090900,137,4,2021-09-10T00:28:10.200,50.0,TB,left,108.61,24.98,0.00,0.00,0.00,168.25,348.56,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.202404,5.691546,4.670514,6.437468,3.002416,9.545287,11.325529,1.973069,19.151214,4.826852,10.685060,0.0,2.976054,21.196323,6.394154,6.397163,16.404786,0.183576,17.431053,16.745059,0.834865,9.906871,1.578892
4,1400,46081.0,2021090900,137,5,2021-09-10T00:28:10.300,50.0,TB,left,108.61,24.98,0.00,0.00,0.00,168.25,337.40,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.105728,5.689868,4.670514,6.416829,2.963866,9.564685,11.257056,1.960255,19.159596,4.818672,10.639140,0.0,2.978590,21.196323,6.381175,6.405279,16.485269,0.165529,17.449837,16.733021,0.866083,9.961390,1.578892
5,1401,46081.0,2021090900,137,6,2021-09-10T00:28:10.400,50.0,TB,left,108.55,24.96,0.00,0.02,0.06,232.84,266.47,autoevent_ballsnap,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.954870,5.637535,4.664386,6.338738,2.991755,9.599818,11.211512,2.022993,19.174384,4.842004,10.619346,0.0,2.978003,21.141433,6.422974,6.368454,16.454777,0.206155,17.407608,16.741858,1.051190,10.030165,1.336899
6,1402,46081.0,2021090900,137,7,2021-09-10T00:28:10.500,50.0,TB,left,108.48,24.99,0.02,0.11,0.07,195.54,277.96,ball_snap,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.845517,5.652690,4.710042,6.286851,2.954810,9.603926,11.235306,2.128098,19.149499,4.899653,10.557107,0.0,3.032903,21.082742,6.416136,6.316011,16.550737,0.540093,17.555184,16.763395,1.194738,10.046039,1.157411
7,1403,46081.0,2021090900,137,8,2021-09-10T00:28:10.600,50.0,TB,left,108.48,24.99,0.03,0.09,0.00,195.54,278.88,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.659290,5.617633,4.710042,6.315069,2.947830,9.666985,11.222999,2.217747,19.160316,4.902418,10.491158,0.0,3.001083,21.082742,6.421970,6.340032,16.605195,0.710634,17.581539,16.833339,1.282225,10.009540,1.543114
8,1404,46081.0,2021090900,137,9,2021-09-10T00:28:10.700,50.0,TB,left,108.46,25.08,0.07,0.18,0.09,121.37,317.69,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.611454,5.679657,4.802208,6.388693,2.862045,9.643236,11.303530,2.257299,19.072378,4.972263,10.365645,0.0,3.119263,20.912764,6.344762,6.310032,16.690356,0.848528,17.658131,16.837161,1.546254,10.048368,2.263382
9,1405,46081.0,2021090900,137,10,2021-09-10T00:28:10.800,50.0,TB,left,108.55,25.23,0.23,0.84,0.18,123.34,11.55,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.640220,5.

In [132]:
# snap to ball release
dlt_play[(dlt_play.time>=dlt_play[dlt_play.event=='ball_snap'][['time']].reset_index().time[0])&(dlt_play.time<=dlt_play[(dlt_play.event=='pass_forward')|(dlt_play.event=='qb_sack')][['time']].reset_index().time[0])]

,index,nflId,gameId,playId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,DRT,ROLB,LT,RLB,LEO,SCBoL,SLWR,RT,RWR,QB,SRoWR,DLT,LG,LCB,SRiWR,SCBiL,RCB,RG,LWR,FS,C,SCBR,dist_from_ball
6,1402,46081.0,2021090900,137,7,2021-09-10T00:28:10.500,50.0,TB,left,108.48,24.99,0.02,0.11,0.07,195.54,277.96,ball_snap,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.845517,5.652690,4.710042,6.286851,2.954810,9.603926,11.235306,2.128098,19.149499,4.899653,10.557107,0.0,3.032903,21.082742,6.416136,6.316011,16.550737,0.540093,17.555184,16.763395,1.194738,10.046039,1.157411
7,1403,46081.0,2021090900,137,8,2021-09-10T00:28:10.600,50.0,TB,left,108.48,24.99,0.03,0.09,0.00,195.54,278.88,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.659290,5.617633,4.710042,6.315069,2.947830,9.666985,11.222999,2.217747,19.160316,4.902418,10.491158,0.0,3.001083,21.082742,6.421970,6.340032,16.605195,0.710634,17.581539,16.833339,1.282225,10.009540,1.543114
8,1404,46081.0,2021090900,137,9,2021-09-10T00:28:10.700,50.0,TB,left,108.46,25.08,0.07,0.18,0.09,121.37,317.69,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.611454,5.679657,4.802208,6.388693,2.862045,9.643236,11.303530,2.257299,19.072378,4.972263,10.365645,0.0,3.119263,20.912764,6.344762,6.310032,16.690356,0.848528,17.658131,16.837161,1.546254,10.048368,2.263382
9,1405,46081.0,2021090900,137,10,2021-09-10T00:28:10.800,50.0,TB,left,108.55,25.23,0.23,0.84,0.18,123.34,11.55,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.640220,5.780138,4.927575,6.604309,2.743465,9.617401,11.452030,2.171543,18.903653,4.968471,10.198167,0.0,3.247845,20.816630,6.178819,6.334390,16.886814,0.917606,17.822225,16.984935,1.811574,10.138170,2.957059
10,1406,46081.0,2021090900,137,11,2021-09-10T00:28:10.900,50.0,TB,left,108.58,25.26,0.40,1.71,0.04,117.95,23.82,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.492005,5.753503,4.952585,6.736468,2.714885,9.682360,11.426294,2.238147,18.835894,5.004018,10.208330,0.0,3.284205,20.808623,6.120988,6.406473,16.994117,1.011385,17.714294,17.078141,1.978004,10.110435,3.618840
11,1407,46081.0,2021090900,137,12,2021-09-10T00:28:11.000,50.0,TB,left,108.60,25.31,0.59,1.94,0.05,116.27,27.64,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.415026,5.661245,4.989589,6.888977,2.660244,9.738444,11.451362,2.289541,18.776456,5.084339,10.194312,0.0,3.351805,20.752393,6.061856,6.421978,17.089836,1.140044,17.742055,17.162765,2.161342,10.098639,4.213704
12,1408,46081.0,2021090900,137,13,2021-09-10T00:28:11.100,50.0,TB,left,108.63,25.37,0.81,2.00,0.07,116.27,31.28,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.301515,5.592254,5.044205,7.106511,2.624290,9.807365,11.455047,2.385330,18.715365,5.193120,10.203842,0.0,3.435025,20.733065,5.979298,6.462600,17.195421,1.278828,17.820180,17.256384,2.335487,10.100317,4.716916
13,1409,46081.0,2021090900,137,14,2021-09-10T00:28:11.200,50.0,TB,left,108.81,25.41,1.52,4.96,0.18,133.46,64.91,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.190016,5.462545,5.053712,7.445865,2.620935,9.982485,11.510156,2.400521,18.666304,5.192302,10.301403,0.0,3.484882,20.788663,5.985825,6.651022,17.425054,1.284562,17.852042,17.555763,2.407862,10.015333,5.007405
14,1410,46081.0,2021090900,137,15,2021-09-10T00:28:11.300,50.0,TB,left,109.00,25.48,2.22,5.54,0.20,130.42,72.18,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.130783,5.372076,5.082696,7.856036,2.600173,10.165599,11.600780,2.368143,18.582755,5.210835,10.399101,0.0,3.537711,20.840518,6.023637,6.897427,17.729343,1.327742,17.974028,17.898282,2.479698,9.987883,5.230574
15,1411,46081.0,2021090900,137,16,2021-09-10T00:28:11.400,50.0,TB,left,109.23,25.55,2.71,4.64,0.25,132.78,75.05,None,Pass Rush,DLT,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.072751,5.20048